Import all libraries

In [11]:
import os
import re
import pandas as pd
import numpy as np 
import tensorflow as tf 

from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, TimeDistributed, Lambda
from tensorflow.keras.models import Model, Sequential
from keras import backend as K

import librosa as lb
from librosa.effects import trim
import librosa.display

from sklearn.model_selection import train_test_split

Setting Up the datasets

In [12]:
dataset_path = "dataset/"
metadata = "Datasets.csv"

audio_directory = "dataset/"

# Create a dataframe for the transcript
dataframe = pd.read_csv(metadata)

# Preprocess transcript
def preprocess_text(text):
    text = text.lower()  # Convert text to uppercase
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove non-alphanumeric characters (except spaces)
    return text

dataframe['clean_transcript'] = dataframe['Transcription'].apply(preprocess_text)

print(dataframe.head())

  File_Path Speaker         Transcription Session      clean_transcript
0     03M_1     03M  1 2 3 4 5 6 7 8 9 10       1  1 2 3 4 5 6 7 8 9 10
1     03M_2     03M                   ata       2                   ata
2     03M_3     03M                   ana       3                   ana
3     03M_4     03M                   ara       4                   ara
4     03M_5     03M                  atha       5                  atha


Data Pre-Processing

In [13]:
# Function to preprocess audio and connect to transcripts
def combine_audio_with_transcript(directory, dataframe):
    audio_data = []    
     # Iterate over each row in the DataFrame
    for index, row in dataframe.iterrows():
        file_name = row['File_Path']  # Get the file name from the CSV (without .wav)
        transcript = row['clean_transcript']  # Get the transcript
        
        # Construct the full file path by combining directory and file name with .wav extension
        file_path = os.path.join(directory, f"{file_name}.wav")
        
        # Check if the file exists in the audio directory
        if os.path.exists(file_path):
            try:
                # Append the processed data along with the transcript
                audio_data.append({
                    "file_path": file_path,
                    "transcript": transcript,
                })
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    
    return audio_data

# Preprocess audio files and connect to transcripts
audio_data_with_transcripts = combine_audio_with_transcript(audio_directory, dataframe)
print(audio_data_with_transcripts)

[{'file_path': 'dataset/01F_1.wav', 'transcript': '1 2 3 4 5 6 7 8 9 10'}, {'file_path': 'dataset/01F_3.wav', 'transcript': 'ana'}, {'file_path': 'dataset/01F_4.wav', 'transcript': 'ara'}]


Prepare dataset

In [14]:
# Get Dataset from the folder

prepared_audio_data = []

# Dataset file path
dataset_file_path = [item['file_path'] for item in audio_data_with_transcripts]

# Transcript
dataset_transcript = [item['transcript'] for item in audio_data_with_transcripts]

max_length = 0
sample_rate = 16000
num_mels = 128

try:
    for file_path in dataset_file_path:
        y, sr = librosa.load(file_path, sr=sample_rate, mono=True)
        temp_length = len(y)
        
        if temp_length > max_length:
            max_length = temp_length
except Exception as e:
    print(f"Error loading {file_path}: {e}")
    
try:
    for file_path in dataset_file_path:
        y, sr = lb.load(file_path, sr=sample_rate, mono=True)
        
        y,_ = lb.effects.trim(y)
        
        y = lb.util.normalize(y)
        
        if len(y) < max_length:
            y = np.pad(y, (0, max_length - len(y)))
        else:
            y = y[:max_length]
            
        # Convert to Mel spectrogram
        mel_spec = lb.feature.melspectrogram(y=y, sr=sr, n_mels=num_mels)
        mel_spec_db = lb.power_to_db(mel_spec, ref=np.max)
        
        # Normalize between 0 and 1
        mel_spec_db = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
        
        prepared_audio_data.append(mel_spec_db.T)
    
        print(f"Loaded {file_path} with shape {y.shape}")
except Exception as e:
    print(f"Error loading {file_path}: {e}")

Loaded dataset/01F_1.wav with shape (148857,)
Loaded dataset/01F_3.wav with shape (148857,)
Loaded dataset/01F_4.wav with shape (148857,)


In [15]:
# Convert to numpy arrays

x_audio = np.array(prepared_audio_data)
y_label = np.array(dataset_transcript)

# Tokenize the transcripts

tokenizer = tf.keras.layers.TextVectorization(output_mode='int')
tokenizer.adapt(y_label)

# Encode transcripts
y_encoded = tokenizer(y_label)

# Vocabulary size (needed for the model output layer)
vocab_size = tokenizer.vocabulary_size()

print(f'Input Shape:  {x_audio}')

Input Shape:  [[[0.54124254 0.5275768  0.53953606 ... 0.01681213 0.         0.        ]
  [0.54170406 0.5462986  0.57169616 ... 0.03955154 0.00250835 0.        ]
  [0.5665705  0.5514461  0.53666604 ... 0.04083033 0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.6441563  0.6819421  0.6368438  ... 0.08405342 0.03869438 0.        ]
  [0.6243746  0.6747502  0.6282508  ... 0.09496956 0.04460564 0.        ]
  [0.6111824  0.6546541  0.6171881  ... 0.06683693 0.01693573 0.        ]
  ...
  [0.7087456  0.6507     0.5734707  ... 0.26553887 0.19489212 0.04593525]
  [0.6978976  0.6383642  0.53054607 ... 0.12152729 0.07639904 0.        ]
  [0.5787447  0.48293296 0.4475916  ... 0.01182508 0.         0.        ]]

 [[0.71273565 0.598681   0.5119035  ... 0.11016979 0.08827572 0.        ]
  [0.724

Set up the model

In [16]:
# Define input shape
input_shape = (None, x_audio.shape[2])  # Variable time steps, fixed features
inputs = Input(shape=input_shape, name="audio_input")

# BiLSTM Layers
x = Bidirectional(LSTM(128, return_sequences=True, name="bilstm_1"))(inputs)
x = BatchNormalization(name="batch_norm_1")(x)
x = Dropout(0.2, name="dropout_1")(x)

x = Bidirectional(LSTM(64, return_sequences=True, name="bilstm_2"))(x)
x = BatchNormalization(name="batch_norm_2")(x)
x = Dropout(0.2, name="dropout_2")(x)

# Output layer with vocabulary size + 1 (for CTC blank token)
output = TimeDistributed(Dense(vocab_size + 1, activation="softmax"), name="output")(x)

# Define standalone ASR model
model = Model(inputs=inputs, outputs=output, name="ASR_BiLSTM_Model")

# CTC Loss Function
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args

    # Ensure input_length and label_length are 1D tensors
    input_length = tf.squeeze(input_length, axis=-1)
    label_length = tf.squeeze(label_length, axis=-1)

    return tf.nn.ctc_loss(
        labels=labels,
        logits=y_pred,
        label_length=tf.cast(label_length, dtype=tf.int32),
        logit_length=tf.cast(input_length, dtype=tf.int32),
        logits_time_major=False
    )

# Additional inputs for CTC loss
labels = Input(name="labels", shape=(None,), dtype="int32")  # Target transcripts
input_length = Input(name="input_length", shape=(1,), dtype="int32")  # Length of input sequences
label_length = Input(name="label_length", shape=(1,), dtype="int32")  # Length of label sequences

# Compute CTC loss using a Lambda layer
ctc_loss = Lambda(ctc_lambda_func, output_shape=(1,), name="ctc_loss")(
    [output, labels, input_length, label_length]
)

# Define the full CTC Model
ctc_model = Model(inputs=[inputs, labels, input_length, label_length], outputs=ctc_loss)

# Compile the model with a dummy loss (CTC loss is computed manually)
ctc_model.compile(optimizer="adam", loss=lambda y_true, y_pred: y_pred)

# Print model summary
ctc_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ audio_input         │ (None, None, 128) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_6     │ (None, None, 256) │    263,168 │ audio_input[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_norm_1        │ (None, None, 256) │      1,024 │ bidirectional_6[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, None, 256) │          0 │ batch_norm_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_7     │ (None, None, 128) │    164,352 │ dropout_1[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_norm_2        │ (None, None, 128) │        512 │ bidirectional_7[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, None, 128) │          0 │ batch_norm_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output              │ (None, None, 15)  │      1,935 │ dropout_2[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ labels (InputLayer) │ (None, None)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_length        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ label_length        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ctc_loss (Lambda)   │ (None, 1)         │          0 │ output[0][0],     │
│                     │                   │            │ labels[0][0],     │
│                     │                   │            │ input_length[0][… │
│                     │                   │            │ label_length[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 430,991 (1.64 MB)

 Trainable params: 430,223 (1.64 MB)

 Non-trainable params: 768 (3.00 KB)

Train Model

In [17]:

# Compute input lengths (number of time steps per audio before padding)
input_lengths = np.array([audio.shape[0] for audio in prepared_audio_data])  # Time steps per sample

# Compute label lengths (number of tokens in each transcript)
label_lengths = np.array([len(label) for label in y_label])  # Tokens per transcript

print(f"Input Lengths: {input_lengths}")
print(f"Label Lengths: {label_lengths}")


history = ctc_model.fit(
    x=[x_audio, y_encoded, input_lengths, label_lengths], 
    y=np.zeros(len(x_audio)),  # Dummy labels for loss
    batch_size=32, 
    epochs=50
)

Input Lengths: [291 291 291]
Label Lengths: [20  3  3]
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step - loss: 980.0148
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - loss: 923.7978
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - loss: 869.5993
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - loss: 834.7711
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - loss: 817.0906
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - loss: 804.5508
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - loss: 790.8588
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - loss: 780.5767
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - loss: 772.4497
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - loss: 767.7099
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - loss: 764.7933
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - loss: 762.5020
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - loss: 761.3952
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - loss: 760.0847
Epo